In [17]:
import sys
sys.path.append('../src')
from data_fetcher import WorldBankAPI
import pandas as pd

wb = WorldBankAPI()

countries = ['USA', 'CHN', 'CAN', 'MEX', 'IND', 'DEU', 'GBR', 'VNM', 'JPN', 'KOR', 'BRA', 'PHL', 'IDN', 'CHE']

indicators = {
    'NE.EXP.GNFS.CD': 'exports',
    'NE.IMP.GNFS.CD': 'imports',
    'TM.TAX.MRCH.WM.AR.ZS': 'tariff_rate',
    'NY.GDP.MKTP.CD': 'gdp',
    'FP.CPI.TOTL.ZG': 'inflation',      # Annual % change in prices
    'NY.GDP.PCAP.CD': 'gdp_per_capita'  # Wealth per person
}

all_data = []

for code, name in indicators.items():
    df = wb.get_indicator(countries, code, start_year=2015, end_year=2024)
    df['metric'] = name  # This adds a label so we don't get the numbers confused
    all_data.append(df)

df_combined = pd.concat(all_data, ignore_index=True)

df_wide = df_combined.pivot_table(
    index=['country_name', 'country_code', 'year'],
    columns='metric',
    values='value'
).reset_index()

df_wide.to_csv('../data/processed/trade_data_clean.csv', index=False)

print("Saved processed data. Final dataset shape:", df_wide.shape)
print(df_wide.head())

Fetching NE.EXP.GNFS.CD for 14 countries...
Retrieved 140 records
Fetching NE.IMP.GNFS.CD for 14 countries...
Retrieved 140 records
Fetching TM.TAX.MRCH.WM.AR.ZS for 14 countries...
Retrieved 140 records
Fetching NY.GDP.MKTP.CD for 14 countries...
Retrieved 140 records
Fetching FP.CPI.TOTL.ZG for 14 countries...
Retrieved 140 records
Fetching NY.GDP.PCAP.CD for 14 countries...
Retrieved 140 records
Saved processed data. Final dataset shape: (140, 9)
metric country_name country_code  year       exports           gdp  \
0            Brazil          BRA  2015  2.324888e+11  1.802212e+12   
1            Brazil          BRA  2016  2.238633e+11  1.795693e+12   
2            Brazil          BRA  2017  2.583307e+11  2.063515e+12   
3            Brazil          BRA  2018  2.805432e+11  1.916934e+12   
4            Brazil          BRA  2019  2.645630e+11  1.873288e+12   

metric  gdp_per_capita       imports  inflation  tariff_rate  
0          8936.195589  2.532727e+11   9.029901         8.27  

In [18]:
df = pd.read_csv('../data/processed/trade_data_clean.csv')
df.columns.name = None
df = df.sort_values(['country_name', 'year'])
df.head(15)

,country_name,country_code,year,exports,gdp,gdp_per_capita,imports,inflation,tariff_rate
0,Brazil,BRA,2015,2.324888e+11,1.802212e+12,8936.195589,2.532727e+11,9.029901,8.27
1,Brazil,BRA,2016,2.238633e+11,1.795693e+12,8836.285460,2.166864e+11,8.739144,8.01
2,Brazil,BRA,2017,2.583307e+11,2.063515e+12,10080.507872,2.435106e+11,3.446373,8.59
3,Brazil,BRA,2018,2.805432e+11,1.916934e+12,9300.660729,2.729944e+11,3.664850,7.95
4,Brazil,BRA,2019,2.645630e+11,1.873288e+12,9029.833044,2.766348e+11,3.732976,7.97
5,Brazil,BRA,2020,2.428721e+11,1.476107e+12,7074.194075,2.339413e+11,3.211768,8.41
6,Brazil,BRA,2021,3.192512e+11,1.670647e+12,7972.536961,3.098533e+11,8.301660,7.76
7,Brazil,BRA,2022,3.831776e+11,1.951924e+12,9281.332821,3.744825e+11,9.280106,7.26
8,Brazil,BRA,2023,3.937316e+11,2.191132e+12,10377.589772,3.439399e+11,4.593563,NaN
9,Brazil,BRA,2024,3.921054e+11,2.185822e+12,10310.548878,3.857089e+11,4.367464,NaN


In [19]:
import os

events_content = """date,phase,event_type,countries_affected,tariff_applied,significance
2025-02-01,Opening Shots,IEEPA,Canada + Mexico,25%,First major move — border justification
2025-04-05,Liberation Day,Universal,All Countries,10%,10% baseline takes effect globally
2025-08-06,Escalation,Secondary Tariff,India,25%,Russian oil penalty
2026-02-06,De-escalation,India Relief,India,0%,Secondary tariff removed
2026-02-07,Partial Deal,US-India Interim,India,18%,Reciprocal tariff reduced to 18%"""

sectors_content = """country_code,sector,us_export_value_usd,tariff_rate,exposure_level
CHN,Electronics,180000000000,25%,CRITICAL
IND,Pharmaceuticals,8500000000,26%,HIGH
CAN,Lumber,7000000000,25%,CRITICAL
MEX,Automotive,100000000000,25%,CRITICAL
DEU,Automobiles,25000000000,15%,HIGH"""

def save_reference_file(filename, content):
    with open(filename, 'w') as f:
        f.write(content)
    print(f"Successfully saved: {filename}")

save_reference_file('../data/reference/trade_war_events.csv', events_content)
save_reference_file('../data/reference/sector_exposure.csv', sectors_content)

print("\n✅ Task 1 Data is now physically inside your files.")

Successfully saved: ../data/reference/trade_war_events.csv
Successfully saved: ../data/reference/sector_exposure.csv

✅ Task 1 Data is now physically inside your files.


In [20]:
events = pd.read_csv('../data/reference/trade_war_events.csv')
sectors = pd.read_csv('../data/reference/sector_exposure.csv')

current_tariffs = {
    'CHN': 47.0, 'CAN': 25.0, 'MEX': 25.0, 'IND': 18.0, 
    'DEU': 15.0, 'GBR': 10.0, 'VNM': 46.0, 'USA': 0.0
}

def apply_2026_context(row):
    if row['year'] == 2024:
        return current_tariffs.get(row['country_code'], row['tariff_rate'])
    return row['tariff_rate']

df_wide['effective_tariff_2026'] = df_wide.apply(apply_2026_context, axis=1)

df_wide.to_csv('../data/processed/trade_war_final_2026.csv', index=False)

print("✅ Analysis Ready! Created 'effective_tariff_2026' column.")
print(df_wide[df_wide['year'] == 2024][['country_code', 'tariff_rate', 'effective_tariff_2026']].head(10))

✅ Analysis Ready! Created 'effective_tariff_2026' column.
metric country_code  tariff_rate  effective_tariff_2026
9               BRA          NaN                    NaN
19              CAN          NaN                   25.0
29              CHN          NaN                   47.0
39              DEU          NaN                   15.0
49              IND          NaN                   18.0
59              IDN          NaN                    NaN
69              JPN          NaN                    NaN
79              KOR          NaN                    NaN
89              MEX          NaN                   25.0
99              PHL          NaN                    NaN


In [21]:
import pandas as pd
import sys
sys.path.append('../src')
from data_fetcher import WorldBankAPI

wb = WorldBankAPI()

country_codes = [
    'USA', 'CHN', 'CAN', 'MEX',   # North American Core
    'IND', 'VNM', 'JPN', 'KOR',   # Asian Manufacturing Hubs
    'DEU', 'GBR', 'CHE',          # European Giants
    'BRA', 'PHL', 'IDN'            # Emerging Markets
]

indicators = {
    'NE.EXP.GNFS.CD': 'exports',
    'NE.IMP.GNFS.CD': 'imports',
    'TM.TAX.MRCH.WM.AR.ZS': 'baseline_tariff',
    'NY.GDP.MKTP.CD': 'gdp',
    'FP.CPI.TOTL.ZG': 'inflation',
    'NE.TRD.GNFS.ZS': 'trade_pct_gdp'
}

all_data = []
for code, name in indicators.items():
    print(f"📡 Fetching {name} for 14 countries...")
    df = wb.get_indicator(country_codes, code, start_year=2020, end_year=2024)
    df['metric'] = name
    all_data.append(df)

df_combined = pd.concat(all_data, ignore_index=True)

print(f"\n✅ Part A Complete: Collected {len(df_combined)} data points.")

📡 Fetching exports for 14 countries...
Fetching NE.EXP.GNFS.CD for 14 countries...
Retrieved 70 records
📡 Fetching imports for 14 countries...
Fetching NE.IMP.GNFS.CD for 14 countries...
Retrieved 70 records
📡 Fetching baseline_tariff for 14 countries...
Fetching TM.TAX.MRCH.WM.AR.ZS for 14 countries...
Retrieved 70 records
📡 Fetching gdp for 14 countries...
Fetching NY.GDP.MKTP.CD for 14 countries...
Retrieved 70 records
📡 Fetching inflation for 14 countries...
Fetching FP.CPI.TOTL.ZG for 14 countries...
Retrieved 70 records
📡 Fetching trade_pct_gdp for 14 countries...
Fetching NE.TRD.GNFS.ZS for 14 countries...
Retrieved 70 records

✅ Part A Complete: Collected 420 data points.


In [22]:
master_data = """country_code,tariff_current_feb2026,tariff_peak,deal_status,special_notes
USA,0.0,0.0,Stable,Domestic market base
CHN,47.0,60.0,Trade War,High-tech restrictions active
CAN,25.0,25.0,Negotiating,Border security link
MEX,25.0,25.0,Negotiating,Auto-sector focus
IND,18.0,50.0,Interim Deal,Reduced from 50% on Feb 7
VNM,46.0,46.0,Trade War,Manufacturing shift target
DEU,15.0,15.0,Stable,EU block baseline
GBR,10.0,10.0,Stable,Special relationship tier
JPN,10.0,10.0,Stable,Tech partnership
KOR,10.0,10.0,Stable,Tech partnership
BRA,8.3,12.0,Neutral,Agricultural exporter
PHL,10.0,10.0,Neutral,Developing nation status
IDN,10.0,10.0,Neutral,Developing nation status
CHE,10.0,10.0,Stable,Neutral finance hub"""

with open('../data/reference/trade_war_master.csv', 'w') as f:
    f.write(master_data)

print("✅ 'trade_war_master.csv' created. Now the codes will match!")

✅ 'trade_war_master.csv' created. Now the codes will match!


In [23]:
tariff_master = pd.read_csv('../data/reference/trade_war_master.csv')

df_final = df_wide.merge(
    tariff_master[['country_code', 'tariff_current_feb2026', 
                    'tariff_peak', 'deal_status', 'special_notes']],
    on='country_code',
    how='left'
)
df_final.to_csv('../data/processed/trade_data_global.csv', index=False)

print(f"Complete dataset: {df_final.shape}")
print(df_final[['country_name', 'year', 'exports', 'tariff_current_feb2026', 'deal_status']].head(20))

Complete dataset: (140, 14)
   country_name  year       exports  tariff_current_feb2026  deal_status
0        Brazil  2015  2.324888e+11                     8.3      Neutral
1        Brazil  2016  2.238633e+11                     8.3      Neutral
2        Brazil  2017  2.583307e+11                     8.3      Neutral
3        Brazil  2018  2.805432e+11                     8.3      Neutral
4        Brazil  2019  2.645630e+11                     8.3      Neutral
5        Brazil  2020  2.428721e+11                     8.3      Neutral
6        Brazil  2021  3.192512e+11                     8.3      Neutral
7        Brazil  2022  3.831776e+11                     8.3      Neutral
8        Brazil  2023  3.937316e+11                     8.3      Neutral
9        Brazil  2024  3.921054e+11                     8.3      Neutral
10       Canada  2015  4.957475e+11                    25.0  Negotiating
11       Canada  2016  4.813576e+11                    25.0  Negotiating
12       Canada  2017  